## Imports

In [9]:
import numpy as np
import pandas as pd
import jsonlines
import gc
import os
import torch
from torch import nn
from models.models import *


from IPython.display import display, clear_output

from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
from transformers.activations import GELUActivation
from transformers.modeling_outputs import MaskedLMOutput
from transformers import DataCollatorForWholeWordMask
from datasets import load_from_disk, load_dataset
from transformers import BertTokenizer, DistilBertTokenizer
from transformers.data.data_collator import _torch_collate_batch
import evaluate
import wandb
from huggingface_hub import notebook_login
#from transformers import PreTrainedModel

import torch
import torch.nn as nn
import torch.nn.functional as F
import math

emb_tsv_file = "wikidata_translation_v1.tsv"

In [10]:
import sys
print(sys.getrecursionlimit())

sys.setrecursionlimit(100000)
# but doing so is dangerous -- the standard limit is a little conservative, but Python stackframes can be quite big.

# import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

100000


In [11]:
bert_model_name = "distilbert-base-uncased" ##"bert-base-cased"

In [12]:
# from transformers import AutoTokenizer, DistilBertForMaskedLM
# import torch

# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# model = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")

# inputs = tokenizer("The capital of France is [MASK].", return_tensors="pt")

# with torch.no_grad():
#     logits = model(**inputs).logits

# # retrieve index of [MASK]
# mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

# predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)

# labels = tokenizer("The capital of France is Paris.", return_tensors="pt")["input_ids"]
# # mask labels of non-[MASK] tokens
# labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

# outputs = model(**inputs, labels=labels)

# Initialize Project

Download:
- embeds_wktxt.csv
- [linked-wikitext-2 dataset](https://rloganiv.github.io/linked-wikitext-2/#/) and unzip

# Tokenization

- `tokens` are the given list of tokens from wikitext2
- `input_ids` are what come from tokenization, they divide certain words into multiple pieces, and each sentence has a CLS and a SEP
- `word_tokens` is the length of `tokens`. For each token in `token`, it mentions how many sub-words it was divided into due to word piece tokenization
- `cummulative_word_tokens` is a cummulative sum of `word_tokens`, with an extra 0 in the beginning

##### Process
index of a token in `token` can be found in `input_ids` by `cummulative_word_tokens`. if `ix` is the index of a word in `token`, its beginning index in `input_ids` is `cummulative_word_tokens[ix] + 1`, the +1 is because `input_ids` has a CLS in the beginning. `token[ix]` spans from `input_ids[cummulative_word_tokens[ix] + 1]` to `input_ids[cummulative_word_tokens[ix+1] + 1]`

In [13]:
embeds_wktxt = pd.read_csv("embeds_wktxt.csv")
qids_wktxt = pd.read_csv("qids_wktxt2.csv")

linked_wikitext_2 = "linked-wikitext-2/"
train = linked_wikitext_2+"train.jsonl"
valid = linked_wikitext_2+"valid.jsonl"
test = linked_wikitext_2+"test.jsonl"

data_files = {"train": train, "valid": valid, "test": test}
wikitest2_dataset = load_dataset("json", data_files=data_files)

chunk_size = 128

class BertTokenizerModified(DistilBertTokenizer): #BertTokenizer
    def __init__(self,vocab_file,**kwargs):
        
        super().__init__(vocab_file, never_split=["@@START@@", "@@END@@", "@@start@@", "@@end@@"], **kwargs)
    
        self.tokenized_list = []

    def _tokenize(self, text):
        token_list = text.split()
        split_tokens = []
        tokenized_list = []
        
        if self.do_basic_tokenize:
            for token in token_list:

                # If the token is part of the never_split set
                if token in self.basic_tokenizer.never_split:
                    split_tokens.append(token)
                    tokenized_list.append(1)
                else:
                    word_tokenized = self.wordpiece_tokenizer.tokenize(token)
                    split_tokens += word_tokenized
                    tokenized_list.append(len(word_tokenized))

        self.tokenized_list.append(tokenized_list)
        return split_tokens
    
def get_cumm(vals):
    cumm = 0
    res = [0] ## len of res is 1 more than vals, with an initial 0
    for val in vals:
        cumm += val
        res.append(cumm)
    return res

def my_tokenize_function(data):
    
    ## tokenize
    my_tokenizer.tokenized_list = []
    result = my_tokenizer([" ".join(eg) for eg in data["tokens"]])
    if my_tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    
    ## save word to token mapping
    ## 3, 1, 1 means the first word got divided into 3 tokens, the next into 1, and the next into 1 again
    result["word_tokens"] = my_tokenizer.tokenized_list
    result["cummulative_word_tokens"] = [get_cumm(x) for x in result["word_tokens"]]
    
    return result

def get_kg_embedding_batched(data):
    
    ## store a masking array that says whether or not an item has kg embedding
    """
    When you specify batched=True the function receives a dictionary with the fields of the dataset, 
    but each value is now a list of values, and not just a single value. 
    """
    input_ids_list = data["input_ids"]
    annotations_list = data['annotations']
    cummulative_word_tokens_list = data["cummulative_word_tokens"]
    
    batch_size = len(input_ids_list)
    embed_list = [] ## len will be batch_size
    embed_mask = []
    embed_mask_qid = []
    
    #add by Edward, the index of qid
    embed_mask_index = []
    
    allc = 0
    cc = 0
    
    for i in range(batch_size):
        input_ids = input_ids_list[i]
        annotations = annotations_list[i]
        
        ## Replace zeros with random numbers if required
        embeds = np.zeros((len(input_ids), 200)) ## CLS, SEP will have np.zeros, like unknown words
        mask = [0]*len(input_ids)
        mask_qid = ['0']*len(input_ids)
        
        #add by Edward
        mask_index = [-100]*len(input_ids)
        
        
        for annot in annotations:
            start_ix, end_ix = annot['span']
            start = cummulative_word_tokens_list[i][start_ix] + 1
            end = cummulative_word_tokens_list[i][end_ix] + 1
            
            qid = annot['id']
            
            #add by Edward
            index_list = qids_wktxt[qids_wktxt["id"]==qid].index.tolist()
            allc += 1
            if len(index_list) == 0:
                qid_index = -100

            else:
                qid_index = index_list[0]
                cc+=1
            
            df = embeds_wktxt[embeds_wktxt['id']==qid]
            if len(df)>0:
                embeds[start:end] = np.tile(df.iloc[0,1:].values.reshape((1,200)),(end-start, 1))
                mask[start:end] = [1]*(end-start)
                mask_qid[start:end] = [qid]*(end-start)
                
                #add by Edward
                mask_index[start:end] = [qid_index]*(end-start)
                
                
        embed_mask.append(mask)
        embed_mask_qid.append(mask_qid)
        embed_list.append(embeds)
        
        #add by Edward
        embed_mask_index.append(mask_index)

    
    print(cc/allc)
    print(cc)
    print(allc)
    return {
        "kg_embedding": embed_list, 
        "kg_embedding_mask": embed_mask,
        "kg_embedding_mask_qid": embed_mask_qid,
        "kg_embedding_mask_index": embed_mask_index
    }

def filter_text_batched(data):
    
    new_data = {k:[] for k in data}
    
    input_ids_list = data["input_ids"]
    
    ## remove [UNK] == 100 
    indices_list = [[i for i,input_id in enumerate(input_ids) if input_id!=100]
                        for input_ids in input_ids_list]
    
    for k in data:
        for indices, data_list in zip(indices_list, data[k]):
            new_data[k].append([data_list[ind] for ind in indices])
        
    return new_data

def truncate_data(data):
    maxlength = my_tokenizer.max_model_input_sizes[bert_model_name]

    ## truncate to maxlength
    for k in data:
        data[k] = [x[:maxlength] for x in data[k]]
    
    return data


def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    
    # Create a new labels column
#     result["labels"] = result["input_ids"].copy()
    return result

Using custom data configuration default-07015f1fc3b429ce
Found cached dataset json (/home/xiangru/.cache/huggingface/datasets/json/default-07015f1fc3b429ce/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
my_tokenizer = BertTokenizerModified.from_pretrained(bert_model_name)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerModified'.


# Data processing and loading

In [15]:
# Linked Wikitext dataset
dataset_file = "concat_dataset_v2"


# final_dataset = wikitest2_dataset.map(my_tokenize_function, batched=True)\
#                           .map(get_kg_embedding_batched, batched=True, batch_size=100, keep_in_memory=False)\
#                           .remove_columns(['title', 'tokens', 'annotations', 'word_tokens', 'cummulative_word_tokens'])\
#                           .map(filter_text_batched, batched=True, batch_size=100, keep_in_memory=False)\
#                           .map(group_texts, batched=True, batch_size=100, keep_in_memory=False)\

# final_dataset.save_to_disk(dataset_file)

In [16]:
# Wikidata fact dataset
synthetic_data = "generate_test_data/sythetic_dataset.jsonl"
synthetic_dataset = load_dataset("json", data_files={"synthetic": synthetic_data})

dataset_file = "tokenized_synthetic_dataset_1"

# tokenized_synthetic_dataset = synthetic_dataset.map(my_tokenize_function, batched=True)\
#                           .map(get_kg_embedding_batched, batched=True, batch_size=100, keep_in_memory=False)\
#                           .remove_columns(['title', 'tokens', 'annotations', 'word_tokens', 'cummulative_word_tokens'])\

# tokenized_synthetic_dataset.save_to_disk(dataset_file)




Using custom data configuration default-90b20df1b3afab02
Found cached dataset json (/home/xiangru/.cache/huggingface/datasets/json/default-90b20df1b3afab02/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
## Load the saved tokenized dataset
dataset_file = "concat_dataset_v2"
final_dataset = load_from_disk(dataset_file)

dataset_file = "tokenized_synthetic_dataset_1"
tokenized_synthetic_dataset = load_from_disk(dataset_file)


# # To test model with smaller sample dataset
train_size = 100
test_size = 300
downsampled_dataset = final_dataset["valid"].train_test_split(train_size=train_size, test_size=test_size, seed=42)

Loading cached split indices for dataset at concat_dataset_v2/valid/cache-f6dda4b0ee1674f7.arrow and concat_dataset_v2/valid/cache-0fc4692bf4cddc18.arrow


In [19]:
#final_dataset['train'][0]

## Create Data Collator for Masking

In [20]:

## Create Data Collator for Masking
from typing import Any, Callable, Dict, List, NewType, Optional, Tuple, Union
from collections.abc import Mapping    
    
# NEW FOR EXTRA TEST    
class CustomDataCollator(DataCollatorForWholeWordMask):
    def torch_call(self, examples: List[Union[List[int], Any, Dict[str, Any]]]) -> Dict[str, Any]:
        if isinstance(examples[0], Mapping):
            input_ids = [e["input_ids"] for e in examples]
            attention_mask = [e["attention_mask"] for e in examples]
            kg_embedding_mask = [e["kg_embedding_mask"] for e in examples]
            kg_embedding = [e["kg_embedding"] for e in examples]
            kg_embedding_mask_qid = [e["kg_embedding_mask_qid"] for e in examples]
            kg_embedding_mask_index = [e["kg_embedding_mask_index"] for e in examples]
        else:
            raise Exception("Dataset needs to be in dictionary format")
 
        batch_input = _torch_collate_batch(input_ids, self.tokenizer, pad_to_multiple_of=self.pad_to_multiple_of)
        #mask_labels = kg_embedding_mask
        batch_mask = _torch_collate_batch(kg_embedding_mask, self.tokenizer, pad_to_multiple_of=self.pad_to_multiple_of)        
        # Edward: Aisha avoided taking tokens that corresponds to entity into the MLM task, why?
        inputs, labels = self.torch_mask_tokens(batch_input, batch_mask)
        
        # by Edward
        batch_attention_mask = _torch_collate_batch(attention_mask, self.tokenizer, pad_to_multiple_of=self.pad_to_multiple_of)
 
        batch_size = len(kg_embedding)
        token_length = len(inputs[0])
        embedding_size = len(kg_embedding[0][0])
 
        kg_embedding = [kg_embds+[[0]*embedding_size]*(token_length-len(kg_embds)) for kg_embds in kg_embedding]
    
        #by Edward
        batch_mask_index = _torch_collate_batch(kg_embedding_mask_index, self.tokenizer, pad_to_multiple_of=self.pad_to_multiple_of)
    
    
    
 
        return {
                "input_ids": inputs, 
                "attention_mask" : batch_attention_mask,
                "labels": labels, 
                "kg_embedding":kg_embedding, 
                "kg_embedding_mask": batch_mask,
                "kg_embedding_mask_qid": [qid+['0']*(token_length-len(qid)) for qid in kg_embedding_mask_qid],
                "kg_embedding_mask_index":batch_mask_index,
               }
    

# Model trial

## Configuration

In [21]:
import os
import copy
import json

def gelu(x):
    """Implementation of the gelu activation function.
        For information: OpenAI GPT's gelu is slightly different (and gives slightly different results):
        0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
    """
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))


def swish(x):
    return x * torch.sigmoid(x)




ACT2FN = {"gelu": gelu, "relu": torch.nn.functional.relu, "swish": swish, "GELU":GELUActivation()}


class BertConfig(object):
    """Configuration class to store the configuration of a `BertModel`.
    """
    def __init__(self,
                 vocab_size_or_config_json_file,
                 hidden_size=768,
                 num_hidden_layers=12,
                 num_attention_heads=12,
                 intermediate_size=3072,
                 hidden_act="GELU",
                 hidden_dropout_prob=0.1,
                 attention_probs_dropout_prob=0.1,
                 max_position_embeddings=512,
                 type_vocab_size=2,
                 initializer_range=0.02,
                 layer_types=None,
                 hidden_size_ent=200,
                 base_model_name="distilbert-base-uncased",
                 kg_vocab_size=46685,
                 kg_num_attention_head=4):
        """Constructs BertConfig.
        Args:
            vocab_size_or_config_json_file: Vocabulary size of `inputs_ids` in `BertModel`.
            hidden_size: Size of the encoder layers and the pooler layer.
            num_hidden_layers: Number of hidden layers in the Transformer encoder.
            num_attention_heads: Number of attention heads for each attention layer in
                the Transformer encoder.
            intermediate_size: The size of the "intermediate" (i.e., feed-forward)
                layer in the Transformer encoder.
            hidden_act: The non-linear activation function (function or string) in the
                encoder and pooler. If string, "gelu", "relu" and "swish" are supported.
            hidden_dropout_prob: The dropout probabilitiy for all fully connected
                layers in the embeddings, encoder, and pooler.
            attention_probs_dropout_prob: The dropout ratio for the attention
                probabilities.
            max_position_embeddings: The maximum sequence length that this model might
                ever be used with. Typically set this to something large just in case
                (e.g., 512 or 1024 or 2048).
            type_vocab_size: The vocabulary size of the `token_type_ids` passed into
                `BertModel`.
            initializer_range: The sttdev of the truncated_normal_initializer for
                initializing all weight matrices.
        """
        if isinstance(vocab_size_or_config_json_file, str):
            with open(vocab_size_or_config_json_file, "r", encoding='utf-8') as reader:
                json_config = json.loads(reader.read())
            for key, value in json_config.items():
                self.__dict__[key] = value
        elif isinstance(vocab_size_or_config_json_file, int):
            self.vocab_size = vocab_size_or_config_json_file
            self.hidden_size = hidden_size
            self.num_hidden_layers = num_hidden_layers
            self.num_attention_heads = num_attention_heads
            self.hidden_act = hidden_act
            self.intermediate_size = intermediate_size
            self.hidden_dropout_prob = hidden_dropout_prob
            self.attention_probs_dropout_prob = attention_probs_dropout_prob
            self.max_position_embeddings = max_position_embeddings
            self.type_vocab_size = type_vocab_size
            self.initializer_range = initializer_range
            self.layer_types = layer_types
            self.hidden_size_ent = hidden_size_ent
            self.base_model_name = base_model_name
            self.kg_vocab_size = kg_vocab_size
            self.kg_num_attention_head = kg_num_attention_head
        else:
            raise ValueError("First argument must be either a vocabulary size (int)"
                             "or the path to a pretrained model config file (str)")

    @classmethod
    def from_dict(cls, json_object):
        """Constructs a `BertConfig` from a Python dictionary of parameters."""
        config = BertConfig(vocab_size_or_config_json_file=-1)
        for key, value in json_object.items():
            config.__dict__[key] = value
        return config

    @classmethod
    def from_json_file(cls, json_file):
        """Constructs a `BertConfig` from a json file of parameters."""
        with open(json_file, "r", encoding='utf-8') as reader:
            text = reader.read()
        return cls.from_dict(json.loads(text))

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"
    
    def to_json_file(self,path):
        with open(path, "w") as outfile:
            outfile.write(self.to_json_string())

## Abstract model

In [22]:
class PreTrainedBertModel(nn.Module):
    """ An abstract class to handle weights initialization and
        a simple interface for dowloading and loading pretrained models.
    """
    def __init__(self, config, *inputs, **kwargs):
        super(PreTrainedBertModel, self).__init__()
        if not isinstance(config, BertConfig):
            raise ValueError(
                "Parameter config in `{}(config)` should be an instance of class `BertConfig`. "
                "To create a model from a Google pretrained model use "
                "`model = {}.from_pretrained(PRETRAINED_MODEL_NAME)`".format(
                    self.__class__.__name__, self.__class__.__name__
                ))
        self.config = config

    def init_bert_weights(self, module):
        """ Initialize the weights.
        """
        if isinstance(module, (nn.Linear, nn.Embedding)):
            # Slightly different from the TF version which uses truncated_normal for initialization
            # cf https://github.com/pytorch/pytorch/pull/5617
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
        elif isinstance(module, BertLayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        if isinstance(module, nn.Linear) and module.bias is not None:
            module.bias.data.zero_()

## Integration module

### Attention block

In [23]:
class BertLayerNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-12):
        """Construct a layernorm module in the TF style (epsilon inside the square root).
        """
        super(BertLayerNorm, self).__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.bias = nn.Parameter(torch.zeros(hidden_size))
        self.variance_epsilon = eps

    def forward(self, x):
        u = x.mean(-1, keepdim=True)
        s = (x - u).pow(2).mean(-1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.variance_epsilon)
        return self.weight * x + self.bias

In [24]:
class BertSelfAttention(nn.Module):
    def __init__(self, config):
        super(BertSelfAttention, self).__init__()
        if config.hidden_size % config.num_attention_heads != 0:
            raise ValueError(
                "The hidden size (%d) is not a multiple of the number of attention "
                "heads (%d)" % (config.hidden_size, config.num_attention_heads))
        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)

        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)

    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(self, hidden_states, attention_mask):
        mixed_query_layer = self.query(hidden_states)
        mixed_key_layer = self.key(hidden_states)
        mixed_value_layer = self.value(hidden_states)

        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(mixed_key_layer)
        value_layer = self.transpose_for_scores(mixed_value_layer)

        # Take the dot product between "query" and "key" to get the raw attention scores.
        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        # Apply the attention mask is (precomputed for all layers in BertModel forward() function)
        attention_scores = attention_scores + attention_mask

        # Normalize the attention scores to probabilities.
        attention_probs = nn.Softmax(dim=-1)(attention_scores)

        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_probs = self.dropout(attention_probs)

        context_layer = torch.matmul(attention_probs, value_layer)
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)
        return context_layer


class BertSelfOutput(nn.Module):
    def __init__(self, config):
        super(BertSelfOutput, self).__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.LayerNorm = BertLayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states


In [25]:
class BertAttention(nn.Module):
    def __init__(self, config):
        super(BertAttention, self).__init__()
        self.self = BertSelfAttention(config)
        self.output = BertSelfOutput(config)

        config_ent = copy.deepcopy(config)
        config_ent.hidden_size = config.hidden_size_ent
        config_ent.num_attention_heads = config.kg_num_attention_head

        self.self_ent = BertSelfAttention(config_ent)
        self.output_ent = BertSelfOutput(config_ent)


    def forward(self, input_tensor, attention_mask, input_tensor_ent, attention_mask_ent):
        self_output = self.self(input_tensor, attention_mask)
        self_output_ent = self.self_ent(input_tensor_ent, attention_mask_ent)
        attention_output = self.output(self_output, input_tensor)
        attention_output_ent = self.output_ent(self_output_ent, input_tensor_ent)
        return attention_output, attention_output_ent
    

class BertIntermediate(nn.Module):
    def __init__(self, config):
        super(BertIntermediate, self).__init__()
        self.dense = nn.Linear(config.hidden_size, config.intermediate_size)
        self.dense_ent = nn.Linear(config.hidden_size_ent, config.intermediate_size)

        #self.dense_1 = nn.Linear(config.hidden_size, 200)
        #self.dense_1_ent = nn.Linear(100, 200)

#         self.intermediate_act_fn = ACT2FN[config.hidden_act] \
#             if isinstance(config.hidden_act, str) else config.hidden_act

        self.intermediate_act_fn = ACT2FN[config.hidden_act]

    def forward(self, hidden_states, hidden_states_ent):
        hidden_states_ = self.dense(hidden_states)
        hidden_states_ent_ = self.dense_ent(hidden_states_ent)

        #hidden_states_1 = self.dense_1(hidden_states)
        #hidden_states_ent_1 = self.dense_1_ent(hidden_states_ent)

        hidden_states = self.intermediate_act_fn(hidden_states_+hidden_states_ent_)
        #hidden_states_ent = self.intermediate_act_fn(hidden_states_1+hidden_states_ent_1)

        return hidden_states#, hidden_states_ent


class BertOutput(nn.Module):
    def __init__(self, config):
        super(BertOutput, self).__init__()
        self.dense = nn.Linear(config.intermediate_size, config.hidden_size)
        self.dense_ent = nn.Linear(config.intermediate_size, config.hidden_size_ent)
        self.LayerNorm = BertLayerNorm(config.hidden_size, eps=1e-12)
        self.LayerNorm_ent = BertLayerNorm(config.hidden_size_ent, eps=1e-12)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, hidden_states_, input_tensor, input_tensor_ent):
        hidden_states = self.dense(hidden_states_)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)

        hidden_states_ent = self.dense_ent(hidden_states_)
        hidden_states_ent = self.dropout(hidden_states_ent)
        hidden_states_ent = self.LayerNorm_ent(hidden_states_ent + input_tensor_ent)

        return hidden_states, hidden_states_ent



In [26]:
class BertLayer(nn.Module):
    def __init__(self, config):
        super(BertLayer, self).__init__()
        self.attention = BertAttention(config)
        self.intermediate = BertIntermediate(config)
        self.output = BertOutput(config)

    def forward(self, hidden_states, attention_mask, hidden_states_ent, attention_mask_ent, ent_mask):
        attention_output, attention_output_ent = self.attention(hidden_states, attention_mask, hidden_states_ent, attention_mask_ent)
        
        
#         print(ent_mask[0])
        
#         for i in range(10):
#             print(attention_output_ent[0][i])
        

        
        attention_output_ent = attention_output_ent * ent_mask.unsqueeze(2)
        
#         for i in range(10):
#             print(attention_output_ent[0][i])

        
        intermediate_output = self.intermediate(attention_output, attention_output_ent)
        layer_output, layer_output_ent = self.output(intermediate_output, attention_output, attention_output_ent)
        # layer_output_ent = layer_output_ent * ent_mask
        return layer_output, layer_output_ent

### Main module

In [27]:
class WordItgtor(nn.Module):
    

    def __init__(self, config):
        super(WordItgtor, self).__init__()
        
        
        self.layer = BertLayer(config)
        

    
    
    def forward(self,  hidden_states, attention_mask, hidden_states_ent, 
                ent_mask,output_all_encoded_layers=False):        

        # We create a 3D attention mask from a 2D tensor mask.
        # Sizes are [batch_size, 1, 1, to_seq_length]
        # So we can broadcast to [batch_size, num_heads, from_seq_length, to_seq_length]
        # this attention mask is more simple than the triangular masking of causal attention
        # used in OpenAI GPT, we just need to prepare the broadcast dimension here.
        extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
        extended_ent_mask = ent_mask.unsqueeze(1).unsqueeze(2)

        # Since attention_mask is 1.0 for positions we want to attend and 0.0 for
        # masked positions, this operation will create a tensor which is 0.0 for
        # positions we want to attend and -10000.0 for masked positions.
        # Since we are adding it to the raw scores before the softmax, this is
        # effectively the same as removing these entirely.
        extended_attention_mask = extended_attention_mask.to(dtype=next(self.parameters()).dtype) # fp16 compatibility
        extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0
        extended_ent_mask = extended_ent_mask.to(dtype=next(self.parameters()).dtype) # fp16 compatibility
        extended_ent_mask = (1.0 - extended_ent_mask) * -10000.0
        
        res_state, res_state_ent = self.layer(hidden_states,
                                      extended_attention_mask,
                                      hidden_states_ent,
                                      extended_ent_mask,
                                      ent_mask)
#         sequence_output = res_state[-1]
#         pooled_output = self.pooler(sequence_output)

        encoded_layers = res_state
        return encoded_layers  #, pooled_output

## Main model

In [28]:


#class BERTModified(nn.Module): #PreTrainedModel
class BERTModified(PreTrainedBertModel): #
    def __init__(self, config):
        
        super().__init__(config) #For PreTrainedModel
        #super().__init__() ## for nn.Module
        
        self.base_model = AutoModel.from_pretrained(self.config.base_model_name)
        self.config = config
        self.kg_size = self.config.kg_vocab_size
                
        self.activation = GELUActivation() # for distilbert
        self.vocab_transform = nn.Linear(self.config.hidden_size, self.config.hidden_size)
        self.vocab_layer_norm = nn.LayerNorm(self.config.hidden_size, eps=1e-12)
        self.vocab_projector = nn.Linear(self.config.hidden_size, self.config.vocab_size)
        
        self.kg_projector = nn.Linear(self.config.hidden_size, self.kg_size)
        
        self.mlm_loss_fct = nn.CrossEntropyLoss()
        
        ## set to eval
        self.base_model.eval()
        
        ## freeze model
        for param in self.base_model.parameters():
            param.requires_grad = False
            
        ## initialization of integrator
#         self.itgt = WordItgtor(self.config.hidden_size,self.config.hidden_size_ent)
        self.itgt = WordItgtor(self.config)

    def forward(
        self,
        kg_embedding = None,           ## given
        kg_embedding_mask = None,      ## given
        kg_embedding_mask_qid = None,      ## given
        kg_embedding_mask_index = None,
        input_ids = None,              ## given
        attention_mask = None,         ## given
        head_mask = None,
        inputs_embeds = None,
        labels = None,                 ## given
        output_attentions = None,
        output_hidden_states = None,
        return_dict= None,):
        
        base_model_output = self.base_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        

        
        ## Get LM embedding
        hidden_states = base_model_output[0]  # (bs, seq_length, dim)
        

        
        ## Use hidden_states and kg_embedding and perform INTEGRATION
        kg_embedding = torch.tensor(kg_embedding).to(device='cuda:0')
        kg_embedding_mask_index = torch.tensor(kg_embedding_mask_index).to(device='cuda:0')
        kg_embedding_mask = torch.tensor(kg_embedding_mask).to(device='cuda:0')
        
        #print(attention_mask)
        
        attention_mask = torch.tensor(attention_mask).to(device='cuda:0')
        
        prediction_logits = self.itgt(hidden_states,attention_mask, kg_embedding, kg_embedding_mask)
#         prediction_logits = self.itgt(hidden_states, kg_embedding)
#         prediction_logits = self.activation(prediction_logits)  # (bs, seq_length, dim)
#         prediction_logits = self.vocab_layer_norm(prediction_logits)  # (bs, seq_length, dim)
    
        lm_prediction_logits = self.vocab_projector(prediction_logits)  # (bs, seq_length, vocab_size)
        kg_prediction_logits = self.kg_projector(prediction_logits)  # (bs, seq_length, kg_size)

        mlm_loss = None
        kg_loss = None
        

        
        if labels is not None:
            mlm_loss = self.mlm_loss_fct(lm_prediction_logits.view(-1, lm_prediction_logits.size(-1)), labels.view(-1))
        if kg_embedding_mask_index is not None:
            kg_loss = self.mlm_loss_fct(kg_prediction_logits.view(-1, kg_prediction_logits.size(-1)), kg_embedding_mask_index.view(-1))

        total_loss = mlm_loss + kg_loss
        
        return MaskedLMOutput(
            loss=total_loss,
            logits=lm_prediction_logits,
            hidden_states=base_model_output.hidden_states,
            attentions=base_model_output.attentions,
        )

# Create Model

1. The KIM designed in this study is BERTModified. 
2. The basiline LM-Raw is BERTModified_LMRaw.
3. The basiline KG-Raw is BERTModified_KGRaw.
4. The alternative integration module Alt-KIM is BERTModified_alt. 

In [29]:
model_name = "Our KIM"  # or LM-Raw, KG-Raw, alt-KIM

model_dict ={"Our KIM":BERTModified, "LM-Raw":BERTModified_LMRaw, "KG-Raw":BERTModified_KGRaw,"alt-KIM":BERTModified_alt}

#base_model = AutoModel.from_pretrained(bert_model_name)


a = BertConfig(30522)
a.to_json_file('sample.json')

config = BertConfig.from_json_file('sample.json')

# choose the model needsto be trained 
# model = model_dict[model_name](bert_model_name = bert_model_name,
#                                   base_model = base_model,
#                                   config = config, kg_size =46685)
model = model_dict[model_name](config)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
# create data_collator
data_collator = CustomDataCollator(tokenizer=my_tokenizer, mlm=True, mlm_probability=0.15)


# Evaluation metrics and loss function

In [31]:
import evaluate

# metrics = evaluate.combine(["accuracy", "precision", "recall", "f1"])
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_preds=None, logits=None, labels=None):
    
    # We should have either `eval_preds` or both `logits` and `labels`
    if eval_preds:
        logits, labels = eval_preds

    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[l for l in label if l != -100] for label in labels]
    true_predictions = [
        [p for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    ## Flatten values
    true_labels = [item for sublist in true_labels for item in sublist]
    true_predictions = [item for sublist in true_predictions for item in sublist]
    
    accuracy = accuracy_metric.compute(predictions=true_predictions, references=true_labels)
    precision = precision_metric.compute(predictions=true_predictions, references=true_labels, average="micro")
    recall = recall_metric.compute(predictions=true_predictions, references=true_labels, average="micro")
    f1 = f1_metric.compute(predictions=true_predictions, references=true_labels, average="micro")
    
    return {
        "precision": precision["precision"],
        "recall": recall["recall"],
        "f1": f1["f1"],
        "accuracy": accuracy["accuracy"],
    }

# Create Trainer

In [32]:
from transformers import TrainingArguments

batch_size = 4

# Show the training loss with every epoch
logging_steps = len(final_dataset['train']) // batch_size #len(final_dataset['train']) // batch_size
model_name = "BERTModified"
output_dir = f"{model_name}-finetuned-wikitext-test"

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
#     fp16=True,
    logging_steps=logging_steps,
    num_train_epochs=50,
#     load_best_model_at_end=True,
#     metric_for_best_model="loss",#metric_name,
#     greater_is_better = False,
    logging_dir='logs',
    report_to="wandb",
#     no_cuda=True,
)

metric_for_best_model (str, optional) — Use in conjunction with load_best_model_at_end to specify the metric to use to compare two different models. Must be the name of a metric returned by the evaluation with or without the prefix "eval_". Will default to "loss" if unspecified and load_best_model_at_end=True (to use the evaluation loss).

If you set this value, greater_is_better will default to True. Don’t forget to set it to False if your metric is better when lower.

In [33]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
#     train_dataset=final_dataset["train"],
#     eval_dataset=final_dataset["valid"],
    train_dataset=final_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/home/xiangru/KGNLP/KG-LM-Integration/BERTModified-finetuned-wikitext-test is already a clone of https://huggingface.co/HideOnBush/BERTModified-finetuned-wikitext-test. Make sure you pull the latest changes with `repo.git_pull()`.


In [34]:
# train_dataset=downsampled_dataset["train"]
# eval_dataset=downsampled_dataset["test"]

# Train Model

In [35]:
# trainer.train()

#training from a checkpoint
trainer.train("./BERTModified-finetuned-wikitext-test/checkpoint-175280")



Loading model from ./BERTModified-finetuned-wikitext-test/checkpoint-175280.
/home/xiangru/miniconda3/envs/kgml/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 17528
  Num Epochs = 50
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 219100
  Number of trainable parameters = 68445855
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 40
  Continuing training from global step 175280
  Will skip the first 40 epochs then the first 0 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip`

0it [00:00, ?it/s]

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: edwardjian. Use `wandb login --relogin` to force relogin


/home/xiangru/miniconda3/envs/kgml/lib/python3.7/site-packages/ipykernel_launcher.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/xiangru/miniconda3/envs/kgml/lib/python3.7/site-packages/ipykernel_launcher.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/xiangru/miniconda3/envs/kgml/lib/python3.7/site-packages/ipykernel_launcher.py:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
41,0.821400,16.566595,0.338112,0.338112,0.338112,0.338112
42,0.794700,16.591980,0.329290,0.329290,0.329290,0.329290
43,0.759000,16.609325,0.333480,0.333480,0.333480,0.333480
44,0.736100,16.686855,0.334142,0.334142,0.334142,0.334142
45,0.709700,16.696791,0.336127,0.336127,0.336127,0.336127
46,0.691200,16.714911,0.334804,0.334804,0.334804,0.334804
47,0.672300,16.600849,0.338553,0.338553,0.338553,0.338553
48,0.661600,16.550951,0.339876,0.339876,0.339876,0.339876
49,0.648700,16.684456,0.333701,0.333701,0.333701,0.333701
50,0.637900,16.619131,0.338774,0.338774,0.338774,0.338774


***** Running Evaluation *****
  Num examples = 300
  Batch size = 4
Saving model checkpoint to BERTModified-finetuned-wikitext-test/checkpoint-179662
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
/home/xiangru/miniconda3/envs/kgml/lib/python3.7/site-packages/ipykernel_launcher.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/xiangru/miniconda3/envs/kgml/lib/python3.7/site-packages/ipykernel_launcher.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/xiangru/miniconda3/envs/kgml/lib/python3.7/site-packages/ipykernel_launcher.py:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clo

TrainOutput(global_step=219100, training_loss=0.14265076324309675, metrics={'train_runtime': 8335.1575, 'train_samples_per_second': 105.145, 'train_steps_per_second': 26.286, 'total_flos': 0.0, 'train_loss': 0.14265076324309675, 'epoch': 50.0})

In [28]:
import math

# basic evluation on perplexity 
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}") #21596 for 1 epoch

eval_results

***** Running Evaluation *****
  Num examples = 300
  Batch size = 4
/home/xiangru/miniconda3/envs/kgml/lib/python3.7/site-packages/ipykernel_launcher.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/xiangru/miniconda3/envs/kgml/lib/python3.7/site-packages/ipykernel_launcher.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/xiangru/miniconda3/envs/kgml/lib/python3.7/site-packages/ipykernel_launcher.py:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


>>> Perplexity: 15699980.03


{'eval_loss': 16.569169998168945,
 'eval_precision': 0.30966034406704895,
 'eval_recall': 0.30966034406704895,
 'eval_f1': 0.30966034406704895,
 'eval_accuracy': 0.30966034406704895,
 'eval_runtime': 16.5226,
 'eval_samples_per_second': 18.157,
 'eval_steps_per_second': 4.539,
 'epoch': 25.0}

# Evaluation

## 1.LM-accuracy

### 1.1 Linked Wikidata

In [29]:
predictions = trainer.predict(final_dataset["test"])

***** Running Prediction *****
  Num examples = 2069
  Batch size = 4
/home/xiangru/miniconda3/envs/kgml/lib/python3.7/site-packages/ipykernel_launcher.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/xiangru/miniconda3/envs/kgml/lib/python3.7/site-packages/ipykernel_launcher.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/xiangru/miniconda3/envs/kgml/lib/python3.7/site-packages/ipykernel_launcher.py:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


KeyboardInterrupt: 

[About micro, macro, weighted precision, recall, f1 for multiclass labels](https://towardsdatascience.com/multi-class-metrics-made-simple-part-ii-the-f1-score-ebe8b2c2ca1)

The following always holds true for the micro-F1 case:

`micro-F1 = micro-precision = micro-recall = accuracy`

In [ ]:
# show the result
ompute_metrics(logits = predictions.predictions, labels = predictions.label_ids)


### 1.2 Wikidata facts

In [ ]:
predictions = trainer.predict(tokenized_synthetic_dataset["synthetic"])

compute_metrics(logits = predictions.predictions, labels = predictions.label_ids)

## 2.KG-accuracy

### 2.1 Linked Wikidata

In [ ]:

#model_name = "Our KIM"
model_dict_kg ={"Our KIM":BERTModified_KG, "KG-Raw":BERTModified_KGRaw,"alt-KIM":BERTModified_altKG}

base_model = AutoModel.from_pretrained(bert_model_name)

# choose the model needsto be trained 
model = model_dict_kg[model_name](bert_model_name = bert_model_name,
                                  base_model = base_model,
                                  config = base_model.config,kg_size =46685)


In [ ]:

checkpoint = torch.load("./" + output_dir + "/pytorch_model.bin")
model.load_state_dict(checkpoint)

In [ ]:


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=final_dataset["train"],
    eval_dataset=final_dataset["valid"],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:

predictions = trainer.predict(final_dataset["test"]) 

labels = np.array(final_dataset["test"]["kg_embedding_mask_index"])

In [ ]:

# show the result
compute_metrics(logits = predictions.predictions, labels =labels)

### 2.2 Wikidata facts

In [ ]:
predictions = trainer.predict(tokenized_synthetic_dataset["synthetic"])
# true label of entitles
labels = np.array(tokenized_synthetic_dataset["synthetic"]["kg_embedding_mask_index"])

# show the result
compute_metrics(logits = predictions.predictions, labels =labels)